<a href="https://colab.research.google.com/github/Lambda-School-Labs/Groa-ds/blob/cortex-development/cortex/testing_w2v_locally.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# testing w2v with multi-user sampled data

In [39]:
import psycopg2
import json
import time
import pandas as pd
from w2v_helpers import * 
import warnings;
warnings.filterwarnings('ignore')

with open('values.json', 'r') as values_file:
    values = json.load(values_file)

connection = psycopg2.connect(
    database  =  values['DB_NAME'],
    user      =  values['DB_USER'],
    password  =  values['DB_PASSWORD'],
    host      =  values['DEV'],
    port      =  values['PORT']
)
try:
    cursor_dog = connection.cursor()
    connection = connection  
    print("Connected!")
except Exception as e:
    print("Connection problem chief!\n")
    print(e)

Connected!


In [0]:
user_id = 1
query = "SELECT date, name, year, letterboxd_uri, rating FROM user_letterboxd_ratings WHERE user_id=%s;"
cursor_dog.execute(query, (user_id,))
ratings_sql= cursor_dog.fetchall()
ratings_sql = pd.DataFrame(ratings_sql, columns = ['Date', 'Name', 'Year', 'Letterboxd URI', 'Rating'])

In [0]:
%%time


id_book = pd.read_csv('title_basics_small.csv')
ratings1 = pd.read_csv('exported_data/letterboxd/riley/ratings.csv').sample(30)
ratings2 = pd.read_csv('exported_data/letterboxd/eric/ratings.csv').sample(30)
ratings3 = pd.read_csv('exported_data/letterboxd/cooper/ratings.csv').sample(20)
ratings4 = ratings_sql.sample(20)
ratings = pd.concat([ratings1, ratings2, ratings3, ratings4]).drop_duplicates()

good_list, bad_list, hist_list, val_list, ratings_dict = prep_data(
                            ratings, id_book, cursor_dog, None, None, good_threshold=3, bad_threshold=2)
rec = Recommender('w2v_limitingfactor_v2.model', id_book, cursor_dog)
w2v_preds = rec.predict(good_list, bad_list, hist_list, val_list, ratings_dict, n=500, harshness=1, rec_movies=True, scoring=True,)

df = pd.DataFrame(w2v_preds, columns = ['Name', 'Year', 'URL', 'Mean Rating', 'Votes', 'Similarity', 'ID'])
filter = (df.Year > 1990) & (df.Votes > 8000) 
print("Number of results: ", len(df[filter]))
df[filter][:20]

Number of results:  104
Wall time: 27.7 s


,Name,Year,URL,Mean Rating,Votes,Similarity,ID
32,Samaritan Girl,2004.0,https://www.imdb.com/title/tt0397619/,7.1,12604.0,0.183208,0397619
43,Short Cuts,1993.0,https://www.imdb.com/title/tt0108122/,7.7,40463.0,0.177187,0108122
46,Stalingrad,1993.0,https://www.imdb.com/title/tt0108211/,7.5,29046.0,0.175091,0108211
48,Hedwig and the Angry Inch,2001.0,https://www.imdb.com/title/tt0248845/,7.7,30428.0,0.174445,0248845
49,Sonatine,1993.0,https://www.imdb.com/title/tt0108188/,7.5,16424.0,0.174093,0108188
55,Recess: School's Out,2001.0,https://www.imdb.com/title/tt0265632/,6.6,8831.0,0.170269,0265632
57,Brat 2,2000.0,https://www.imdb.com/title/tt0238883/,7.8,11135.0,0.169078,0238883
70,Natural Born Killers,1994.0,https://www.imdb.com/title/tt0110632/,7.3,205835.0,0.164091,0110632
75,Nixon,1995.0,https://www.imdb.com/title/tt0113987/,7.1,27560.0,0.159682,0113987
81,Basquiat,1996.0,https://www.imdb.com/title/tt0115632/,6.9,18660.0,0.157195,0115632


In [0]:
    user_id=11
    query = "SELECT date, name, year FROM user_groa_willnotwatchlist WHERE user_id=%s;"
    cursor_dog.execute(query, (user_id,))
    watched_sql= cursor_dog.fetchall()
    willnotwatchlist_df = pd.DataFrame(watched_sql, columns = ['Date', 'Name', 'Year'])

Testing Predictor.py locally

In [0]:
from predictor import PythonPredictor 

In [0]:
predictor = PythonPredictor(config)

In [0]:
config = {"bucket": "groa2", "key": "models/w2v.zip"}

In [0]:
sample = {
"user_id": 11,
"number_of_recommendations": 50,
"good_threshold": 5,
"bad_threshold": 4,
"harshness": 1
}

In [44]:
predictor.predict(sample)

Connected!
user_id not found in Letterboxd ratings
user_id not found in Letterboxd reviews
Recommendation committed to DB with id: 75f9b2776290dabf412de2ef1ce6e192


{'recommendation1': '75f9b2776290dabf412de2ef1ce6e192',
 'result1': '[{"Title": "Forrest Gump", "Year": 1994, "IMDB URL": "https://www.imdb.com/title/tt0109830/", "Mean Rating": 8.8, "Votes": 1682162, "Similarity": 0.6394564509391785, "ID": "0109830", "Gem": false}, {"Title": "Goodfellas", "Year": 1990, "IMDB URL": "https://www.imdb.com/title/tt0099685/", "Mean Rating": 8.7, "Votes": 948302, "Similarity": 0.6372131109237671, "ID": "0099685", "Gem": false}, {"Title": "Saving Private Ryan", "Year": 1998, "IMDB URL": "https://www.imdb.com/title/tt0120815/", "Mean Rating": 8.6, "Votes": 1156650, "Similarity": 0.6334736943244934, "ID": "0120815", "Gem": false}, {"Title": "The Usual Suspects", "Year": 1995, "IMDB URL": "https://www.imdb.com/title/tt0114814/", "Mean Rating": 8.5, "Votes": 936523, "Similarity": 0.6329244375228882, "ID": "0114814", "Gem": false}, {"Title": "A Clockwork Orange", "Year": 1971, "IMDB URL": "https://www.imdb.com/title/tt0066921/", "Mean Rating": 8.3, "Votes": 71446